In [1]:
import pandas as pd
import plotly.graph_objects as go
import numpy as np

In [2]:
# df_solar_new = pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/basic_files/solar_total_production.csv")
# df_solar_new_forecast= pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/basic_files/solar_wind_forecast.csv") 

df_solar_new = pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
# df_solar_new = pd.read_csv("C:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_total_production.csv")
# df_solar_new_forecast= pd.read_csv("C:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/basic_files/solar_wind_forecast.csv") 

In [3]:
# df_solar = pd.read_csv("//srvditz1/home$/hohensteinpa/Desktop/Test/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")
df_solar = pd.read_csv("D:/Users/paulh/Desktop/Domäneprojekt2/Energy_production_price_prediction/HEFTcom24/data/solar1.csv")

In [4]:
df_solar_new.generation_mw = df_solar_new.generation_mw / 2

In [5]:
df_solar_new.generation_mw.describe()

count    71403.000000
mean       123.955330
std        200.105899
min          0.000000
25%          0.000000
50%          0.268581
75%        184.404914
max        983.542940
Name: generation_mw, dtype: float64

In [6]:
df_solar.drop(columns=['Unnamed: 0','boa_MWh','Wind_MW','Wind_MWh_credit'], inplace=True)
df_solar.valid_time = pd.to_datetime(df_solar.valid_time) 
df_solar.reference_time = pd.to_datetime(df_solar.reference_time)
df_solar = df_solar.groupby("valid_time").last().reset_index()

In [7]:
# Extracting hour of the day from 'valid_time'
df_solar['hour'] = df_solar['valid_time'].dt.hour

# Sine and cosine encoding for hour (for cyclical behavior)
df_solar['sin_hour'] = np.sin(2 * np.pi * df_solar['hour'] / 24)
df_solar['cos_hour'] = np.cos(2 * np.pi * df_solar['hour'] / 24)

# Day of the year (seasonality)
df_solar['day_of_year'] = df_solar['valid_time'].dt.dayofyear

# Sine and cosine encoding for day of the year (for cyclical seasonality)
df_solar['sin_day'] = np.sin(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['cos_day'] = np.cos(2 * np.pi * df_solar['day_of_year'] / 365)
df_solar['Mean_SolarRadiation_dwd'] = df_solar[[f'SolarDownwardRadiation_Point{i}_dwd' for i in range(20)]].mean(axis=1)
df_solar['Mean_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].mean(axis=1)
df_solar['Std_Temperature_dwd'] = df_solar[[f'Temperature_Point{i}_dwd' for i in range(7)]].std(axis=1)
df_solar["SolarDownwardRadiation_RW_dwd_Mean_30min"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 1).mean()
df_solar["SolarDownwardRadiation_RW_dwd_Mean_1h"] = df_solar["Mean_SolarRadiation_dwd"].rolling(window= 2).mean()
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_30min"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 1)
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_1h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 2)
df_solar["SolarDownwardRadiation_dwd_Mean_Lag_24h"] = df_solar["Mean_SolarRadiation_dwd"].shift(periods= 48)


In [8]:
def pv_temperature_efficiency(irradiance, ambient_temp, NOCT=45, wind_speed=1, eta_0=0.18, beta=0.004):
    # Calculate cell temperature using the simplified NOCT model
    Tc = ambient_temp + (NOCT - 20) * (irradiance / 800)
    
    # Calculate the efficiency loss due to increased cell temperature
    efficiency = eta_0 * (1 - beta * (Tc - 25))
    
    return Tc, efficiency

In [9]:
temperature_columns = [
 'Temperature_Point0_dwd',
 'Temperature_Point1_dwd',
 'Temperature_Point2_dwd',
 'Temperature_Point3_dwd',
 'Temperature_Point4_dwd',
 'Temperature_Point5_dwd',
 'Temperature_Point6_dwd',
 'Temperature_Point7_dwd',
 'Temperature_Point8_dwd',
 'Temperature_Point9_dwd',
 'Temperature_Point10_dwd',
 'Temperature_Point11_dwd',
 'Temperature_Point12_dwd',
 'Temperature_Point13_dwd',
 'Temperature_Point14_dwd',
 'Temperature_Point15_dwd',
 'Temperature_Point16_dwd',
 'Temperature_Point17_dwd',
 'Temperature_Point18_dwd',
 'Temperature_Point19_dwd',
]

irradiance_columns = [
    'SolarDownwardRadiation_Point0_dwd', 'SolarDownwardRadiation_Point1_dwd',
    'SolarDownwardRadiation_Point2_dwd', 'SolarDownwardRadiation_Point3_dwd',
    'SolarDownwardRadiation_Point4_dwd', 'SolarDownwardRadiation_Point5_dwd',
    'SolarDownwardRadiation_Point6_dwd', 'SolarDownwardRadiation_Point7_dwd',
    'SolarDownwardRadiation_Point8_dwd', 'SolarDownwardRadiation_Point9_dwd',
    'SolarDownwardRadiation_Point10_dwd', 'SolarDownwardRadiation_Point11_dwd',
    'SolarDownwardRadiation_Point12_dwd', 'SolarDownwardRadiation_Point13_dwd',
    'SolarDownwardRadiation_Point14_dwd', 'SolarDownwardRadiation_Point15_dwd',
    'SolarDownwardRadiation_Point16_dwd', 'SolarDownwardRadiation_Point17_dwd',
    'SolarDownwardRadiation_Point18_dwd', 'SolarDownwardRadiation_Point19_dwd',
]
for i in range(20):
    for source in ['dwd']:
        temp_col = f'Temperature_Point{i}_{source}'
        irradiance_col = f'SolarDownwardRadiation_Point{i}_{source}'
        panel_temp_col = f'Panel_Temperature_Point{i}_{source}'
        panel_eff_col = f'Panel_Efficiency_Point{i}_{source}'
        
        df_solar[panel_temp_col], df_solar[panel_eff_col] = zip(*df_solar.apply(
            lambda row: pv_temperature_efficiency(row[irradiance_col], row[temp_col]), axis=1))

In [10]:
df_solar["Panel_Temperature_dwd_mean"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").mean(axis= 1)
df_solar["Panel_Efficiency_dwd_mean"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").mean(axis= 1)
df_solar["Panel_Temperature_dwd_std"] = df_solar.filter(regex= r"Panel_Temperature.*_dwd").std(axis= 1)
df_solar["Panel_Efficiency_dwd_std"] = df_solar.filter(regex= r"Panel_Efficiency.*_dwd").std(axis= 1)

In [11]:
df_solar_new.timestamp_utc = pd.to_datetime(df_solar_new.timestamp_utc)

In [13]:
df_solar_new.timestamp_utc = pd.to_datetime(df_solar_new.timestamp_utc) 
merged_df = pd.merge(df_solar_new, df_solar, left_on='timestamp_utc',right_on='valid_time', how='inner')

In [14]:
merged_df.Solar_MWh_credit = merged_df.Solar_MWh_credit / merged_df.capacity_mwp
merged_df["solar_mw_lag_48h"] = merged_df.Solar_MWh_credit.shift(periods= 96)
merged_df["capacity_mwp_lag_48h"] = merged_df.capacity_mwp.shift(periods= 96)
merged_df["Target_Capacity_MWP%"] = merged_df.generation_mw / merged_df.capacity_mwp
merged_df["Target_Capacity_MWP%_lag_48h"] = merged_df["Target_Capacity_MWP%"].shift(periods= 96)


In [15]:
#remove rows from november 21 2022 til december 8 2022
merged_df = merged_df[(merged_df['timestamp_utc'] < '2022-11-21') | (merged_df['timestamp_utc'] > '2022-12-08')]

In [19]:
df_solar1 = merged_df[[ "timestamp_utc",
    "Mean_SolarRadiation_dwd",
    "SolarDownwardRadiation_RW_dwd_Mean_30min",
    "SolarDownwardRadiation_RW_dwd_Mean_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_30min",
    "SolarDownwardRadiation_dwd_Mean_Lag_1h",
    "SolarDownwardRadiation_dwd_Mean_Lag_24h",
    "Panel_Efficiency_dwd_mean",
    "Panel_Efficiency_dwd_std",
    "Panel_Temperature_dwd_mean",
    "Panel_Temperature_dwd_std",
    "Std_Temperature_dwd",
    "Mean_Temperature_dwd",
    "cos_hour",
    "cos_day","solar_mw_lag_48h","capacity_mwp_lag_48h","Target_Capacity_MWP%_lag_48h",
    "Target_Capacity_MWP%","Solar_MWh_credit"]]

In [20]:
df_solar1.columns.shape

(17,)

In [21]:
df_solar1 = df_solar1.dropna(axis=0)

In [22]:
df_solar1.capacity_mwp_lag_48h.mean()

2217.375139835068

In [23]:
df_solar1.columns

Index(['Mean_SolarRadiation_dwd', 'SolarDownwardRadiation_RW_dwd_Mean_30min',
       'SolarDownwardRadiation_RW_dwd_Mean_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_30min',
       'SolarDownwardRadiation_dwd_Mean_Lag_1h',
       'SolarDownwardRadiation_dwd_Mean_Lag_24h', 'Panel_Efficiency_dwd_mean',
       'Panel_Efficiency_dwd_std', 'Panel_Temperature_dwd_mean',
       'Panel_Temperature_dwd_std', 'Std_Temperature_dwd',
       'Mean_Temperature_dwd', 'cos_hour', 'cos_day', 'solar_mw_lag_48h',
       'capacity_mwp_lag_48h', 'Solar_MWh_credit'],
      dtype='object')

In [24]:
df_solar1.to_csv("train.csv", index= False)

In [25]:
train = pd.read_csv("train.csv")
train

,Mean_SolarRadiation_dwd,SolarDownwardRadiation_RW_dwd_Mean_30min,SolarDownwardRadiation_RW_dwd_Mean_1h,SolarDownwardRadiation_dwd_Mean_Lag_30min,SolarDownwardRadiation_dwd_Mean_Lag_1h,SolarDownwardRadiation_dwd_Mean_Lag_24h,Panel_Efficiency_dwd_mean,Panel_Efficiency_dwd_std,Panel_Temperature_dwd_mean,Panel_Temperature_dwd_std,Std_Temperature_dwd,Mean_Temperature_dwd,cos_hour,cos_day,solar_mw_lag_48h,capacity_mwp_lag_48h,Solar_MWh_credit
0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188614,0.000551,13.036279,0.765701,0.803106,13.006348,1.000000,-0.133015,0.0,2130.749758,0.0
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188661,0.000563,12.971021,0.781760,0.825109,12.943795,1.000000,-0.133015,0.0,2130.749758,0.0
2,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188708,0.000577,12.905762,0.800989,0.848471,12.881243,0.965926,-0.133015,0.0,2130.749758,0.0
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188702,0.000595,12.914154,0.826301,0.880315,12.888415,0.965926,-0.133015,0.0,2130.749758,0.0
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.188696,0.000616,12.922546,0.855915,0.915270,12.895586,0.866025,-0.133015,0.0,2130.749758,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
50026,0.009818,0.009818,0.004048,-0.001721,-0.003729,0.004615,0.187994,0.000419,13.896639,0.581458,0.717884,13.779720,0.965926,-0.591261,0.0,2454.186979,0.0
50027,0.021356,0.021356,0.015587,0.009818,-0.001721,0.017088,0.188246,0.000398,13.547243,0.552118,0.658103,13.422141,1.000000,-0.577292,0.0,2454.186979,0.0
50028,0.012875,0.012875,0.017116,0.021356,0.009818,-0.000814,0.188469,0.000386,13.237374,0.536205,0.593122,13.101896,1.000000,-0.577292,0.0,2454.186979,0.0
50029,0.004395,0.004395,0.008635,0.012875,0.021356,-0.018716,0.188692,0.000381,12.927505,0.529482,0.534463,12.781651,0.965926,-0.577292,0.0,2454.186979,0.0


In [26]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
# Assuming df_solar1 is already defined
# df_solar1 = ...

# Split the data into features and target
X = df_solar1.drop(columns=["Solar_MWh_credit"]).values
y = df_solar1["Solar_MWh_credit"].values

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Standardize the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)
# save the scaler
import pickle
with open('scaler.pkl', 'wb') as f:
    pickle.dump(scaler, f)

In [27]:
X_train, X_val, y_train, y_val = train_test_split(X_train_scaled, y_train, test_size=0.2, random_state=42)

In [28]:
# import torch
# import torch.nn as nn

# # Define the MLP model
# class MLP(nn.Module):
#     def __init__(self, input_dim):
#         super(MLP, self).__init__()
#         self.fc1 = nn.Linear(input_dim, 64)
#         self.fc2 = nn.Linear(64, 256)
#         self.fc3 = nn.Linear(256, 512)
#         self.fc4 = nn.Linear(512, 64)
#         self.fc5 = nn.Linear(64, 9)

#         self.dropout = nn.Dropout(0.2)
#         self.swish = nn.SiLU()
    
#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = self.dropout(x)
#         x = self.swish(self.fc2(x))
#         x = self.dropout(x)
#         x = self.swish(self.fc3(x))
#         x = self.dropout(x)
#         x = self.swish(self.fc4(x))
#         x = self.dropout(x)
#         x = self.fc5(x)
#         return x

In [29]:
import torch
import torch.nn as nn

# Define the MLP model
class MLP(nn.Module):
    def __init__(self, input_dim):
        super(MLP, self).__init__()
        # Layer sizes from the best trial
        layer_sizes = [256, 448, 192, 96]
        dropout_rates = [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]

        # Define the layers
        self.fc1 = nn.Linear(input_dim, layer_sizes[0])
        self.fc2 = nn.Linear(layer_sizes[0], layer_sizes[1])
        self.fc3 = nn.Linear(layer_sizes[1], layer_sizes[2])
        self.fc4 = nn.Linear(layer_sizes[2], layer_sizes[3])
        self.fc5 = nn.Linear(layer_sizes[3], 9)  # Output layer

        # Dropouts
        self.dropout1 = nn.Dropout(dropout_rates[0])
        self.dropout2 = nn.Dropout(dropout_rates[1])
        self.dropout3 = nn.Dropout(dropout_rates[2])
        self.dropout4 = nn.Dropout(dropout_rates[3])

        # Activation function (Swish)
        self.swish = nn.SiLU()
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))  # First layer with ReLU
        x = self.dropout1(x)         # First dropout
        x = self.swish(self.fc2(x))  # Second layer with Swish
        x = self.dropout2(x)         # Second dropout
        x = self.swish(self.fc3(x))  # Third layer with Swish
        x = self.dropout3(x)         # Third dropout
        x = self.swish(self.fc4(x))  # Fourth layer with Swish
        x = self.dropout4(x)         # Fourth dropout
        x = self.fc5(x)              # Output layer (no activation for raw outputs)
        return x



In [30]:
# Best trial:
#   Value: 9.009602808952332
#   Params:
#     batch_size: 512
#     n_layers: 4
#     layer_0_size: 256
#     layer_1_size: 448
#     layer_2_size: 192
#     layer_3_size: 96
#     dropout_0: 0.12338360578207397
#     dropout_1: 0.2192742565593194
#     dropout_2: 0.15708417985889997
#     dropout_3: 0.253419888887539
#     learning_rate: 0.001699486730428881
# Layer sizes: [256, 448, 192, 96]
# Dropout rates: [0.12338360578207397, 0.2192742565593194, 0.15708417985889997, 0.253419888887539]
# Number of layers: 4

In [31]:
# Convert to PyTorch tensors
X_train_tensor = torch.tensor(X_train_scaled, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32).view(-1, 1)
X_test_tensor = torch.tensor(X_test_scaled, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.float32).view(-1, 1)
X_val_tensor = torch.tensor(X_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32).view(-1, 1)

# Create DataLoader
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), 
                             torch.tensor(y_train, dtype=torch.float32).view(-1, 1))
train_loader = DataLoader(train_dataset, batch_size=1024, shuffle=True)



# Initialize the model, loss function, and optimizer
input_dim = X_train_scaled.shape[1]
model = MLP(input_dim)
optimizer = optim.Adam(model.parameters(), lr=0.0001)

In [32]:
def pinball_loss(y_true, y_pred, quantiles):
    errors = y_true - y_pred
    loss = torch.max((quantiles - 1) * errors, quantiles * errors)
    return torch.mean(loss)

In [33]:
quantiles = torch.tensor([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])

# Training parameters
num_epochs = 500
patience = 15  # Number of epochs to wait for improvement
min_relative_improvement = 0.000001  # 0.1% improvement threshold
best_val_loss = float('inf')
early_stopping_counter = 0

# Lists to store metrics
train_losses = []
val_losses = []
test_losses = []

for epoch in range(num_epochs):
    # Training phase
    model.train()
    epoch_loss = 0
    for X_batch, y_batch in train_loader:
        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = pinball_loss(y_batch, outputs, quantiles)
        loss.backward()
        optimizer.step()
        epoch_loss += loss.item()
    
    avg_train_loss = epoch_loss / len(train_loader)
    train_losses.append(avg_train_loss)
    
    # Validation phase
    model.eval()
    with torch.no_grad():
        val_outputs = model(X_val_tensor)
        val_loss = pinball_loss(y_val_tensor, val_outputs, quantiles).item()
        val_losses.append(val_loss)
        
        test_outputs = model(X_test_tensor)
        test_loss = pinball_loss(y_test_tensor, test_outputs, quantiles).item()
        test_losses.append(test_loss)
    
    # Early stopping and relative improvement check
    if val_loss < best_val_loss:
        relative_improvement = (best_val_loss - val_loss) / best_val_loss
        if relative_improvement < min_relative_improvement and epoch > 10:
            print(f"Stopping due to small relative improvement: {relative_improvement:.6f}")
            break
        
        best_val_loss = val_loss
        early_stopping_counter = 0
        # Save best model
        # torch.save(model.state_dict(), 'best_model.pth')
        torch.save(model.state_dict(), 'best_model.pth')

    else:
        early_stopping_counter += 1
    
    # Early stopping check
    if early_stopping_counter >= patience:
        print(f"Early stopping triggered after {epoch + 1} epochs")
        break
    
    # Print progress
    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}], "
              f"Train Loss: {avg_train_loss:.4f}, "
              f"Val Loss: {val_loss:.4f}, "
              f"Test Loss: {test_loss:.4f}")

# Load best model after training
model.load_state_dict(torch.load('best_model.pth'))

# Final evaluation
model.eval()
with torch.no_grad():
    final_test_outputs = model(X_test_tensor)
    final_test_loss = pinball_loss(y_test_tensor, final_test_outputs, quantiles).item()
print(f"Final Test Loss: {final_test_loss:.4f}")

Epoch [10/500], Train Loss: 0.0073, Val Loss: 0.0049, Test Loss: 0.0050
Epoch [20/500], Train Loss: 0.0063, Val Loss: 0.0045, Test Loss: 0.0046
Epoch [30/500], Train Loss: 0.0057, Val Loss: 0.0044, Test Loss: 0.0045
Epoch [40/500], Train Loss: 0.0054, Val Loss: 0.0042, Test Loss: 0.0043
Epoch [50/500], Train Loss: 0.0051, Val Loss: 0.0041, Test Loss: 0.0042
Epoch [60/500], Train Loss: 0.0049, Val Loss: 0.0041, Test Loss: 0.0042
Epoch [70/500], Train Loss: 0.0047, Val Loss: 0.0040, Test Loss: 0.0041
Epoch [80/500], Train Loss: 0.0046, Val Loss: 0.0040, Test Loss: 0.0041
Epoch [90/500], Train Loss: 0.0046, Val Loss: 0.0040, Test Loss: 0.0040
Epoch [100/500], Train Loss: 0.0045, Val Loss: 0.0039, Test Loss: 0.0040
Epoch [110/500], Train Loss: 0.0044, Val Loss: 0.0039, Test Loss: 0.0040
Epoch [120/500], Train Loss: 0.0044, Val Loss: 0.0039, Test Loss: 0.0040
Epoch [130/500], Train Loss: 0.0044, Val Loss: 0.0039, Test Loss: 0.0040
Epoch [140/500], Train Loss: 0.0044, Val Loss: 0.0038, Test 

C:\Users\paulh\AppData\Local\Temp\ipykernel_10828\2939562789.py:70: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('best_model.pth'))


In [34]:
import torch

# Assuming your trained model is called 'model'

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation (since we're only making predictions)
with torch.no_grad():
    # Get model predictions for X_test_tensor
    y_pred_tensor = model(X_test_tensor)  # Shape: (num_samples, 9) for 9 quantiles

# Convert y_test_tensor to numpy for easier comparison
y_true = y_test_tensor.numpy()

# Extract the 50% quantile from predictions (assuming it's the 5th quantile at index 4)
y_pred = y_pred_tensor[:, 4].numpy()

# Now y_true contains the true test values and y_pred contains the 50% quantile predictions


In [35]:
len(y_pred)

10007

In [36]:
das

NameError: name 'das' is not defined

In [37]:
import torch
import plotly.graph_objects as go

# Assuming your trained model is called 'model'

# Set the model to evaluation mode
model.eval()

# Disable gradient calculation (since we're only making predictions)
with torch.no_grad():
    # Get model predictions for X_test_tensor
    y_pred_tensor = model(X_test_tensor)  # Shape: (num_samples, 9) for 9 quantiles

# Convert y_test_tensor to numpy for easier comparison
y_true = y_test_tensor.numpy().flatten()  # Flatten to 1D array for plotting

# Extract the 50% quantile from predictions (assuming it's the 5th quantile at index 4)
y_pred = y_pred_tensor[:, 4].numpy().flatten()  # Flatten to 1D array for plotting

# Now y_true contains the true test values and y_pred contains the 50% quantile predictions

# Create the line plot using Plotly
fig = go.Figure()

# Add real values (y_true) to the plot
fig.add_trace(go.Scatter(x=list(range(len(y_true))), 
                         y=y_true, 
                         mode='lines+markers', 
                         name='True Values'))

# Add 50% quantile predictions (y_pred) to the plot
fig.add_trace(go.Scatter(x=list(range(len(y_pred))), 
                         y=y_pred, 
                         mode='lines+markers', 
                         name='50% Quantile Predictions'))

# Update layout
fig.update_layout(title='True Values vs 50% Quantile Predictions',
                  xaxis_title='Index',
                  yaxis_title='Value',
                  template='plotly')

# Show the plot
fig.show()


In [38]:
# Plot the results with Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=list(range(1, num_epochs+1)), y=train_losses, mode='lines', name='Train Loss'))
fig.add_trace(go.Scatter(x=list(range(1, num_epochs+1)), y=test_losses, mode='lines', name='Test Loss'))

fig.update_layout(title='Training and Test Loss Over Epochs',
                  xaxis_title='Epoch',
                  yaxis_title='Loss')

fig.show()

In [42]:
# Plot the predictions vs. true targets with Plotly
y_pred_numpy = y_pred
y_test_numpy = y_test_tensor.numpy()

fig = go.Figure()
fig.add_trace(go.Scatter(x=y_test_numpy.flatten(), y=y_pred_numpy.flatten(), mode='markers', name='Predictions'))

fig.update_layout(title='Predictions vs. True Targets',
                  xaxis_title='True Targets',
                  yaxis_title='Predictions')

fig.show()

In [43]:
# average error
np.mean(np.abs(y_pred_numpy.flatten() - y_test_numpy.flatten()))

0.009617844

In [85]:
2597.3718/512

5.072991796875

In [130]:
mae = np.mean(np.abs(y_true - y_pred))
print(f"Mean Absolute Error (MAE): {mae:.4f}")

# Create scatter plot (y_true vs y_pred)
fig = go.Figure()

# Add scatter plot for true vs predicted values
fig.add_trace(go.Scatter(x=y_true, 
                         y=y_pred, 
                         mode='markers', 
                         name='Predicted vs True'))

# Add a line y = x for reference (perfect prediction line)
fig.add_trace(go.Scatter(x=y_true, 
                         y=y_true, 
                         mode='lines', 
                         name='y = x', 
                         line=dict(color='red', dash='dash')))

# Update layout
fig.update_layout(title='Scatter Plot: True Values vs Predicted (50% Quantile)',
                  xaxis_title='True Values',
                  yaxis_title='Predicted Values',
                  template='plotly')

# Show the plot
fig.show()

Mean Absolute Error (MAE): 20.8100


In [44]:
correlation_coefficient = np.corrcoef(y_true, y_pred)[0, 1]
correlation_coefficient

0.970902073536103

In [48]:
#pinnball loss for each quantile
quantiles = [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9]
for i in range(9):
    errors = y_true - y_pred_tensor[:, i].numpy().flatten()
    loss = np.mean(np.maximum((quantiles[i] - 1) * errors, quantiles[i] * errors))
    print(f"Pinball Loss (Quantile {quantiles[i]}): {loss*2217.375139835068:.4f}")

Pinball Loss (Quantile 0.1): 5.0467
Pinball Loss (Quantile 0.2): 7.7723
Pinball Loss (Quantile 0.3): 9.4585
Pinball Loss (Quantile 0.4): 10.3866
Pinball Loss (Quantile 0.5): 10.6632
Pinball Loss (Quantile 0.6): 10.3361
Pinball Loss (Quantile 0.7): 9.3675
Pinball Loss (Quantile 0.8): 7.7261
Pinball Loss (Quantile 0.9): 5.1391


In [ ]:
2217.375139835068

In [136]:
# Save the model's state dictionary
model_path = 'model.pth'  # Choose your file path and name
torch.save(model.state_dict(), model_path)

In [132]:
X_scaled = scaler.transform(X)
X_tensor = torch.tensor(X_scaled, dtype=torch.float32)
predictions = model(X_tensor)

In [133]:
merged_df = merged_df.dropna(axis=0)

In [134]:
for i in range(9):
    merged_df[f"quantile_{i}"] = predictions[:, i].to('cpu').detach().numpy()

In [135]:
merged_df.to_csv("solar_bidding.csv", index= False)